## Задача 3-2. Задача TSP: нижняя оценка Гельда—Карпа.

В этой задаче Вам предлагается релизовать алгоритм Гельда—Карпа для нижней оценки стоимости решения в задаче Euclidean TSP.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP. Это в точности те же входные данные, что и в задании 3-1.
* Реализуйте функцию `lower_bound_tsp`. При этом можно пользоваться каким-нибудь стандартным алгоритмом построения минимального остовного дерева из библиотеки [`networkx`](https://networkx.github.io/), входящей в состав дистрибутива Anaconda.
* Запустите функцию `run_all()`, чтобы протестировать свой код, и напишите полученные, как следствия, верхние оценки погрешностей решений, которые были получены Вашими алгоритмами NN и NI при решении задания 3-1. Запишите свои выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [1]:
from typing import List, Tuple
from math import sqrt
from itertools import combinations, islice
import numpy as np
import time
from os.path import exists
import networkx as nx

In [2]:
def read_tsp_instance(filename: str) -> List[Tuple[int,int]]:
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0, 0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates


def euclidean_distance(point1: Tuple[int,int], point2: Tuple[int,int]) -> float:
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)

In [63]:
def lower_bound_tsp(vertex_coordinates: List[Tuple[int,int]]) -> float:
    # Replace this trivial lower bound with Held—Karp:
    #return sum(islice(
    #    sorted(euclidean_distance(a, b) for a, b in combinations(vertex_coordinates, 2)), 

    # Не добавляем ребра из нулевой вершины в граф G
    G = nx.Graph()
    for i in range(1, len(vertex_coordinates)):
        for j in range(1, len(vertex_coordinates)):
            G.add_edge(i, j, 
                       weight=euclidean_distance(vertex_coordinates[i], vertex_coordinates[j]))
    T = nx.minimum_spanning_tree(G)
    for iter in range(10, 20):
        weights = np.zeros_like(G.nodes())
        for node in G.nodes():
            weights[node - 1] = 1/(iter ** 10) * (2 - G.degree(node)) 
        for i in range(1, len(vertex_coordinates)):
            for j in range(1, len(vertex_coordinates)):
                G[i][j]['weight'] -= weights[i - 1] + weights[j - 1]
        T = nx.minimum_spanning_tree(G)
    
    min1 = G[1][2]['weight']
    min2 = G[1][2]['weight']
    for node in G.nodes():
        if G[1][node]['weight'] < min1:
            min1 = G[1][node]['weight']
        elif G[1][node]['weight'] < min2:
            min2 = G[1][node]['weight']
    ans = min1 + min2
    for node_from in T.nodes():
        for node_to in T.nodes():
            ans += G[node_from][node_to]['weight']
    return ans

In [64]:
def run_all():
    instance_filenames = ['pr107.tsp', 'pr152.tsp', 'd198.tsp', 'pr439.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        bound = lower_bound_tsp(instance)
        time_nn = time.monotonic()-time_start
        print(' done in {:.2} seconds with lower bound {}'.format(time_nn, int(bound)))

In [ ]:
run_all()

Instance pr107.tsp… done in 1.8 seconds with lower bound 60012247
Instance pr152.tsp… done in 3.7 seconds with lower bound 155975883
Instance d198.tsp… done in 6.3 seconds with lower bound 36583613
Instance pr439.tsp… done in 4.3e+01 seconds with lower bound 829427222
Instance d493.tsp… done in 5.9e+01 seconds with lower bound 217874531
Instance d657.tsp…

## Выводы
Для d198.tsp длина цикла, найденного алгоритмом Nearest Neighbour составляла 18266, Nearest Insertion - 17722, а длина, которую дает нижняя оценка Хельда-Карпа - 36583613, что не очень хорошо, ведь это нижняя оценка, то есть она не больше реального ответа и не меньше, чем $\frac{2}{3} * len(tour)$. Значит, алгоритмы NN и NI дают очень большую погрешность (величины отличаются на несколько порядков) 